<a href="https://colab.research.google.com/github/4may/MIResearch/blob/master/MI_survey_at_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIの最新技術調査

右も左もわからん状態でやり始めると、何やってるのかよくわからん・・・となる。特に、ドメイン知識がない分野だと以下のstep1とstep4がお手上げ状態。

1. 課題を決める
2. 解決方法を決める
3. 解く
4. 評価

そこで、いったんsurveyを読むことで、MIにはどんな課題があって、どんな方法が好まれてて、何を基準に評価したらいいのか、みたいなことを知る。

[Recent advances and applications of machine learning in solid-
state materials science](https://www.nature.com/articles/s41524-019-0221-0)

2019年に提出されたやつ。読むぞ。参考文献の数500はヤバイ笑

## Introduction

* 従来の材料開発では、**人の手による実験**が行われていた
  * 材料開発の実験は、**コスパが悪い**。投資した時間やお金に対して、調べられる材料の数が少ない。
  * 一部には、人の**カン**頼みなところがあった。
* PCを使う材料開発手法の登場
  * **DFT(Density Functional Theory)**
  * monte calro
  * molecular dynamics
  * これらのおかげで、材料開発はいくらかコスパが良くなった。
* PCの性能発展により、high-throughput計算が可能になった。
  * high-throughput計算とは、高度にシステム化された実験環境において、短期間に大量の化合物を評価して、新規の化合物を発見する手法のこと。大体、2,3ヶ月かけて、数万から数十万単位の評価を行う。
  * [参考](https://www.jstage.jst.go.jp/article/kagakutoseibutsu1962/38/4/38_4_264/_pdf)
* **機械学習アルゴリズム**の登場
  * 材料開発での応用事例
    * 安定な材料の予測。特に、熱電、光電材料
    * 材料特性の計算
    * 第一原理計算の高速化
    * リチウムイオン電池
    * 原子シミュレーション
* MIにおける機械学習の利用は**まだ始まったばかり**
  * 以下のようなことが原因で、機械学習の良さを引き出せていない
    * 材料データの数が少ない
    * 材料データのプロパティ数(特徴量)の数が少ない
* 機械学習アルゴリズムは、**ブラックボックス**。なぜ上手くいってるかが良くわからない。
* 特に、固体材料を中心に調べていく

## 機械学習の基本原理

* MIでは、**教師あり学習**を使うことが多い
* MIにおけるcross validation
  * leave-one-out CVは避けるべき。構造と特性の関係を評価するには不十分なため。
  * 未知の材料探索においては、k-fold CVも**楽観的すぎる評価**となるのでいまいち。
  * [**leave-one-cluster-out CV**](https://pubs.rsc.org/en/content/articlelanding/2018/me/c8me00012c#!divAbstract)は、material scienceのために開発された評価方法
    * 機械学習モデルが、未知の材料データをどれだけ推定できるかを計る指標
* leave-one-cluster-outについて
  * アルゴリズム

  1. 材料データをk個のクラスタに分割する
  2. 1個のクラスタを予測のために除外する
  3. 残りのk-1個のクラスタを使って学習する
  4. step2のクラスタを使って予測する

  step1のkの決め方は以下の通り。
  
  * kを決めるためのアルゴリズム

  1. 入力データをnormalizeする
  2. 以下、n回繰り返す.nはCVの回数
    
    * k-meansのセントロイド初期化が上手くいくように、データをシャッフルする
    * k-meansをk=2からk=10まで行う。10としたのは、k fold CVにおいて、10が上限値とされることが多いため。
    * 以下、k個に分かれているクラスタごとに以下を行う。どのkが一番良い結果を生むか調べる。

      * 任意のクラスタを一つ、データから除外する
      * 残りのk-1個のクラスタから成るデータを使って、学習する
      * 除外したクラスタを使って、予測する
      * 各k毎に結果の平均値と標準偏差をまとめる

  * なお、k-meansでいろんなkを試す代わりに、X-meansやG-meansを使って最適なkを計算しても良い。この場合、2-1 ~ 2.3.4のk-meansを使う複数の処理を一つの処理に置き換えられることになる。

### Databases

* データを公平に扱うのが大事
  * 4つの原則

  1. findability
  2. accessibility
  3. interoperability(相互運用性) : いろんな企業、機関が使える。
  4. repurposability(再利用性) : 別の目的(材料開発以外)に使える。

* メタ情報やフォーマットを揃えるのが理想
* 上手くいかなかった例も共有しよう
  * 機械学習の分野では、上手くいったパターンのみが公開されがち。
  * 失敗は成功のもと！
  * 車輪の再失敗を防ぐためにも、袋小路は見つけたら教えよう

### Features

* 材料開発における特徴量とは、**異なる原子や結晶構造を区別するための情報**である
* **記述子(descriptors)** : 化学式を数値に変換したもの。この数値は材料特性を踏まえて作成される。(ただ、1から順に番号を振るだけではない)。数値は、数十〜数百の数字で構成される。
  * 記述子には相関がない方が良い。大量の相関があると、次元の呪いに陥る可能性がある。
  * descriptorの計算方法には、いくつか種類がある。
  * pythonだと、[matminer](https://hackingmaterials.lbl.gov/matminer/#data-descriptor-tools)や[xenonpy](https://ma.issp.u-tokyo.ac.jp/app/1434)を使って計算できる。xenonpyのdescriptor計算は、基本的にmatminerのラッパーとなっている。
  * 基本的には、原子の動きから計算する。原子の動きとは、rotation、reflection、tranlation、permutationを指す。
  * Crystal Graph Convolutional Neural Network(CGCNN)を使って、結晶のdescriptorを計算する方法もある。
  * 記述子を使った学習は、いい成果出してるみたい
* matminerは[別のノートブック](https://github.com/4may/MIResearch/blob/master/matminer.ipynb)で遊んでみよう

* 最近の論文@JAP
  * ベイズ最適化多いね
  * [Machine-learning-assisted thin-film growth: Bayesian optimization in molecular beam epitaxy of SrRuO3 thin films](https://arxiv.org/pdf/1908.00739.pdf)
  * [Materials informatics based on evolutionary algorithms: Application to search for superconducting hydrogen compounds](https://arxiv.org/pdf/1908.00746.pdf)
  * [Synergy of Binary Substitutions for Improving the Cycle Performance in LiNiO2 Revealed by Ab Initio Materials Informatics](https://arxiv.org/pdf/2001.02876.pdf)